In [210]:
from bs4 import BeautifulSoup
from splinter import Browser
from selenium import webdriver
import pandas as pd
import time
import sys
import json
import dateparser 
import requests
import lxml
from lxml import html
from lxml import etree
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

## Storing the Data

In [321]:
#We will be using a dictionary that resembles our main table. 
restaurants = {}
restaurants["names"] = []
restaurants["cusine"] = []
restaurants["price"] = []
restaurants["reviews_count"] = []
restaurants["rating"] = []
restaurants["address"] = []
restaurants["url"] = []



In [319]:
restaurants

{'names': [],
 'cusine': [],
 'price': [],
 'reviews_count': [],
 'rating': [],
 'address': [],
 'url': []}

In [262]:
def go_next_first_time():
    """Function to go from one restaurant to the next in Google Maps using Keyboard Control"""
    N = 9  # number of times you want to press TAB

    actions = ActionChains(browser) 
    #Press Tab Key
    actions.send_keys(Keys.TAB * N)
    actions.send_keys(Keys.SPACE)
    actions.perform()
time.sleep(3)

In [263]:
def go_next():
    """Function to go from one restaurant to the next in Google Maps using Keyboard Control"""
    N = 2  # number of times you want to press TAB

    actions = ActionChains(browser) 
    #Press Tab Key
    actions.send_keys(Keys.TAB * N)
    actions.send_keys(Keys.SPACE)
    actions.perform()
time.sleep(3)

In [293]:
def scrape_rest_info():
    restaurants["names"].append(browser.find_element_by_class_name("x3AX1-LfntMc-header-title-title").text)
    restaurants["cusine"].append(browser.find_element_by_xpath("""//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/span[1]/span[1]/button""").text)
    try:
        restaurants["price"].append(browser.find_elements_by_xpath("""//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span[2]/span[2]/span[1]/span""")[0].text)
    except IndexError:
        restaurants["price"].append("Not Available")
    
    try:
        restaurants["rating"].append(browser.find_element_by_class_name("aMPvhf-fI6EEc-KVuj8d").text)
    except:
        restaurants["rating"].append("Not Available")
    restaurants["reviews_count"].append(browser.find_element_by_class_name("Yr7JMd-pane-hSRGPd").text)
    restaurants["address"].append(browser.find_elements_by_class_name("QSFF4-text")[0].text)
    restaurants["url"].append(browser.current_url)

## Access Google Maps

In [322]:
#Getting the base url
## Links in Google maps can be customized through queries: https://developers.google.com/maps/documentation/urls/get-started
###Forming the Search URL -> https://www.google.com/maps/search/?api=1&parameters
####Based on this, our base url is https://www.google.com/maps/search/?api=1&query=restaurants+in+london+uk
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get('https://www.google.com/maps/search/?api=1&query=restaurants+in+london+uk')
time.sleep(5)
#The first pop up we get is to accept or customize cookies. Here, we will simply accept them by 
##clicking on the xpath of the 'agree' button
browser.find_element_by_xpath('//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[4]/form/div/div/button').click()
browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[7]/div/a').click()
time.sleep(3)


#restaurants["address"].append(browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[9]/div[1]/button/div[1]/div[2]/div[1]').text)
counter = 0
page = 1
while page < 6:
    if counter == 0 and page == 1:
        scrape_rest_info()
        time.sleep(10)
        #Change focus to lower pane
        browser.find_element_by_xpath('//*[@id="sGi9mc-m5SR9c-bottom-pane"]/div/div[1]/div/div/div/div[2]/div[1]').click()
        go_next_first_time()
        counter += 1
    elif counter == 0 and page > 1:
        time.sleep(5)
        #Click on first restaurant below ads
        browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[7]/div/a').click()
        time.sleep(3)
        #Get restaurant info
        scrape_rest_info()
        #Change focus to lower pane
        browser.find_element_by_xpath('//*[@id="sGi9mc-m5SR9c-bottom-pane"]/div/div[1]/div/div/div/div[2]/div[1]').click()
        go_next_first_time()
        counter += 1
        time.sleep(5)
    elif counter < 18 and page > 0:
        go_next()
    ###All Restaurants after the first###
    
        scrape_rest_info()
        time.sleep(5)
        counter += 1
    elif counter == 18 and page > 0:
        #Go back
        browser.find_element_by_xpath('//*[@id="omnibox-singlebox"]/div[1]/div[1]/button').click()
    #Go to next page
        time.sleep(5)
        browser.find_element_by_xpath('//*[@id="ppdPk-Ej1Yeb-LgbsSe-tJiF1e"]').click()
    #Start over
        counter = 0
        page += 1
    

browser.close()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/marianaarrieta/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache
/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """
/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.
/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # This is added back by InteractiveShellApp.init_path()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=98.0.4758.109)
Stacktrace:
0   chromedriver                        0x0000000108481ee9 chromedriver + 5013225
1   chromedriver                        0x000000010840d1d3 chromedriver + 4534739
2   chromedriver                        0x0000000107fe3a68 chromedriver + 170600
3   chromedriver                        0x0000000107fc510e chromedriver + 45326
4   chromedriver                        0x0000000108038dbc chromedriver + 519612
5   chromedriver                        0x000000010804770f chromedriver + 579343
6   chromedriver                        0x00000001080356d3 chromedriver + 505555
7   chromedriver                        0x000000010800b76e chromedriver + 333678
8   chromedriver                        0x000000010800c745 chromedriver + 337733
9   chromedriver                        0x000000010843defe chromedriver + 4734718
10  chromedriver     